In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, RandomSampler
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

from model import Mnist_classifer

In [10]:
#get mnist data

batch = 128

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

mnist = datasets.MNIST(root='C:/Users/weiso131/Desktop/Gans_mnist/data', train=True, download=True, transform=transform)

indice = torch.randperm(len(mnist))

train_sampler = RandomSampler(indice[:int(0.9 * len(indice))])
val_sampler = RandomSampler(indice[int(0.9 * len(indice)):int(0.95 * len(indice))])
test_sampler = RandomSampler(indice[int(0.95 * len(indice)):])

train_dataloader = DataLoader(mnist, batch_size=batch, sampler=train_sampler)
val_dataloader = DataLoader(mnist, batch_size=batch, sampler=val_sampler)
test_dataloader = DataLoader(mnist, batch_size=batch, sampler=test_sampler)



In [11]:
model = Mnist_classifer()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss_function = nn.CrossEntropyLoss()

use_gpu = torch.cuda.is_available()

if (use_gpu):
    print("use GPU")
    model = model.to(device="cuda", dtype=torch.float32)
    loss_function = loss_function.to(device="cuda", dtype=torch.float32)

epoch = 0


use GPU


In [12]:
def validate(model, loss_function, dataloader, use_gpu : bool):
    model.eval()
    val_loss = 0
    val_acc = 0
    count = 0
    with torch.no_grad():
        for x, y in dataloader:
            count += 1
            if (use_gpu):
                x = x.to(device="cuda", dtype=torch.float32)
                y = y.to(device="cuda", dtype=torch.int64)
            #計算
            predict = model(x)
            
            

            #計算loss
            loss =  loss_function(predict, y)     
            val_loss += loss.item()
            
            #計算acc
            topk, top_class = predict.topk(1, dim=1)
            y = y.view(top_class.shape)
            val_acc += int((top_class == y).sum()) / len(y)
    return val_loss / count, val_acc / count
                

In [13]:
min_val_loss = 1e9

for i in range(epoch):
    model.train()
    train_loss = 0
    train_acc = 0
    count = 0
    
    for x, y in train_dataloader:
        count += 1
        optimizer.zero_grad()
        if (use_gpu):
            x = x.to(device="cuda", dtype=torch.float32)
            y = y.to(device="cuda", dtype=torch.int64)
        #計算
        predict = model(x)
        
        #計算loss
        loss =  loss_function(predict, y)     
        train_loss += loss.item()

        #計算acc
        topk, top_class = predict.topk(1, dim=1)
        y = y.view(top_class.shape)
        
        train_acc += int((top_class == y).sum()) / len(y)
        
        #反向傳播
        loss.backward()
        optimizer.step()

    train_loss, train_acc = train_loss / count , train_acc / count
    val_loss, val_acc = validate(model, loss_function, val_dataloader, use_gpu)
    #儲存最佳模型
    if (val_loss < min_val_loss):
        min_val_loss = val_loss
        checkpoint = model.state_dict()
        torch.save(checkpoint, "checkpoint.pth")
        print("save model")
        
    print(f"train loss: {train_loss}, train acc: {train_acc}, val_loss: {val_loss}, val_acc: {val_acc}")

        
    

In [14]:
checkpoint = torch.load("checkpoint.pth")
model.load_state_dict(checkpoint)

test_loss, test_acc = validate(model, loss_function, train_dataloader, use_gpu)
print(f"test loss: {test_loss}, test acc: {test_acc}")

test loss: 0.002598424886329486, test acc: 0.9990928613744076
